#Flood Prediction

Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Telemetri Data

In [ ]:
def gsheet_to_csv(url: str):
  sheet_url = url
  csv = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
  return csv

df_telemetri_rainfall = pd.read_csv(gsheet_to_csv("https://docs.google.com/spreadsheets/d/1nI8m27noE1mMiXQXde8jyXD6-qhuMQ2tE-gXxBkuxi4/edit#gid=0"))
df_telemetri_rainfall

,Location,Date,Time,Rainfall,Status
0,Sumur Batu,15-Jul-20,14:00,0.00 mm,Cerah
1,Sumur Batu,15-Jul-20,15:00,0.00 mm,Cerah\
2,Sumur Batu,16-Jul-20,7:00,0.00 mm,Cerah
3,Sumur Batu,17-Jul-20,4:00,0.00 mm,Cerah
4,Sumur Batu,17-Jul-20,6:00,0.00 mm,Cerah
...,...,...,...,...,...
182880,Cimanggu,28-Jan-21,3:00,0.00 mm,Cerah
182881,Cimanggu,28-Jan-21,5:00,0.00 mm,Cerah
182882,Cimanggu,28-Jan-21,7:00,0.00 mm,Cerah
182883,Cimanggu,28-Jan-21,8:00,12.00 mm,Lebat


In [ ]:
df_telemetri_waterlevel = pd.read_csv(gsheet_to_csv("https://docs.google.com/spreadsheets/d/1xy16th0oBqQ9kux8XGKmkk6AO1flGq1hj1kYqDLn4YI/edit#gid=0"))
df_telemetri_waterlevel.rename(columns = {'Water Level (cm)': 'water_level', 'Date ':'Date'}, inplace = True)
df_telemetri_waterlevel

,Location,Date,Time,water_level
0,Cileungsi,2-Jun-19,0:00,34 cm
1,Cileungsi,2-Jun-19,1:00,34 cm
2,Cileungsi,2-Jun-19,2:00,34 cm
3,Cileungsi,2-Jun-19,3:00,34 cm
4,Cileungsi,2-Jun-19,4:00,34 cm
...,...,...,...,...
129208,Pondok Kelapa,22 Jan 2021,5:00,-188 cm
129209,Pondok Kelapa,22 Jan 2021,6:00,-188 cm
129210,Pondok Kelapa,22 Jan 2021,7:00,-188 cm
129211,Pondok Kelapa,22 Jan 2021,8:00,-188 cm


In [ ]:
# Pandas Left Join is Out of Memory
# pd.merge(df_telemetri_rainfall, df_telemetri_waterlevel, how="left", on="Location")
# Let's try to use sql

**Pandas Left Join is Out of Memory**
```python
pd.merge(df_telemetri_rainfall, df_telemetri_waterlevel, how="left", on="Location")
```
Let's try to use sql instead

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///./db.sqlite3', echo=False)

In [ ]:
# Save to sql
df_telemetri_waterlevel.to_sql("telemetri_waterlevel", if_exists="replace", con=engine)
df_telemetri_rainfall.to_sql("telemetri_rainfall", if_exists="replace", con=engine)

In [ ]:
# check length
print(len(df_telemetri_rainfall))
print(len(df_telemetri_waterlevel))

182885
129213


Join operations ini untuk menggabungkan data dengan cara mengambil semua data yang memiliki nilai **water_level** dan **Rainfall** (jika ada label data banjir dari PUPR)
```
SELECT 
  tr.Date AS Date,
  tr.Time AS Time,
  tr.Location AS Location,
  tr.Rainfall AS Rainfall,
  tr.Status AS RainfallStatus,
  tw.water_level AS WaterLevel
FROM telemetri_rainfall AS tr
LEFT JOIN telemetri_waterlevel AS tw
ON tr.Location = tw.Location AND tr.Date = tw.Date AND tr.Time = tw.Time

UNION

SELECT 
  tw.Date AS Date,
  tw.Time AS Time,
  tw.Location AS Location,
  tr.Rainfall AS Rainfall,
  tr.Status AS RainfallStatus,
  tw.water_level AS WaterLevel
FROM telemetri_waterlevel AS tw
LEFT JOIN telemetri_rainfall AS tr
ON tr.Location = tw.Location AND tr.Date = tw.Date AND tr.Time = tw.Time
```
Namun karena menggunakan asumsi bahwa jika **RainfallStatus** == lebat  sebagai label banjir maka digunakan join ini
```
SELECT 
  tr.Date AS Date,
  tr.Time AS Time,
  tr.Location AS Location,
  tr.Rainfall AS Rainfall,
  tr.Status AS RainfallStatus,
  tw.water_level AS WaterLevel
FROM telemetri_rainfall AS tr
LEFT JOIN telemetri_waterlevel AS tw
ON tr.Location = tw.Location AND tr.Date = tw.Date AND tr.Time = tw.Time
```

In [ ]:
query = engine.execute('''
SELECT 
  tr.Date AS Date,
  tr.Time AS Time,
  tr.Location AS Location,
  tr.Rainfall AS Rainfall,
  tr.Status AS RainfallStatus,
  tw.water_level AS WaterLevel
FROM telemetri_rainfall AS tr
LEFT JOIN telemetri_waterlevel AS tw
ON tr.Location = tw.Location AND tr.Date = tw.Date AND tr.Time = tw.Time
''')
df_tmrain_tmwater = pd.DataFrame(query.fetchall())
df_tmrain_tmwater.columns = query.keys()
df_tmrain_tmwater

,Date,Time,Location,Rainfall,RainfallStatus,WaterLevel
0,15-Jul-20,14:00,Sumur Batu,0.00 mm,Cerah,None
1,15-Jul-20,15:00,Sumur Batu,0.00 mm,Cerah\,None
2,16-Jul-20,7:00,Sumur Batu,0.00 mm,Cerah,None
3,17-Jul-20,4:00,Sumur Batu,0.00 mm,Cerah,None
4,17-Jul-20,6:00,Sumur Batu,0.00 mm,Cerah,None
...,...,...,...,...,...,...
182880,28-Jan-21,3:00,Cimanggu,0.00 mm,Cerah,None
182881,28-Jan-21,5:00,Cimanggu,0.00 mm,Cerah,None
182882,28-Jan-21,7:00,Cimanggu,0.00 mm,Cerah,None
182883,28-Jan-21,8:00,Cimanggu,12.00 mm,Lebat,None


In [ ]:
df_tmrain_tmwater.describe()

,Date,Time,Location,Rainfall,RainfallStatus,WaterLevel
count,182885,182882,182885,182882,182882,9368
unique,1221,24,19,297,9,20
top,11-Sep-20,12:00,Bendungan Gintung,0.00 mm,Cerah,"9,690 cm"
freq,384,7776,19218,173749,173669,2558


In [ ]:
# Asumsi Labeling
df_tmrain_tmwater['Banjir'] = df_tmrain_tmwater.apply(lambda row: 1 if row['RainfallStatus'] != None and row['RainfallStatus'].lower() == "lebat" else 0, axis=1)
df_tmrain_tmwater

,Date,Time,Location,Rainfall,RainfallStatus,WaterLevel,Banjir
0,15-Jul-20,14:00,Sumur Batu,0.00 mm,Cerah,None,0
1,15-Jul-20,15:00,Sumur Batu,0.00 mm,Cerah\,None,0
2,16-Jul-20,7:00,Sumur Batu,0.00 mm,Cerah,None,0
3,17-Jul-20,4:00,Sumur Batu,0.00 mm,Cerah,None,0
4,17-Jul-20,6:00,Sumur Batu,0.00 mm,Cerah,None,0
...,...,...,...,...,...,...,...
182880,28-Jan-21,3:00,Cimanggu,0.00 mm,Cerah,None,0
182881,28-Jan-21,5:00,Cimanggu,0.00 mm,Cerah,None,0
182882,28-Jan-21,7:00,Cimanggu,0.00 mm,Cerah,None,0
182883,28-Jan-21,8:00,Cimanggu,12.00 mm,Lebat,None,1


In [ ]:
# Berapa banjir ya?
sum(df_tmrain_tmwater['Banjir'])

1036

COBA TRAINING

In [ ]:
df_tmrain_tmwater

,Date,Time,Location,Rainfall,RainfallStatus,WaterLevel,Banjir
0,15-Jul-20,14:00,Sumur Batu,0.00 mm,Cerah,None,0
1,15-Jul-20,15:00,Sumur Batu,0.00 mm,Cerah\,None,0
2,16-Jul-20,7:00,Sumur Batu,0.00 mm,Cerah,None,0
3,17-Jul-20,4:00,Sumur Batu,0.00 mm,Cerah,None,0
4,17-Jul-20,6:00,Sumur Batu,0.00 mm,Cerah,None,0
...,...,...,...,...,...,...,...
182880,28-Jan-21,3:00,Cimanggu,0.00 mm,Cerah,None,0
182881,28-Jan-21,5:00,Cimanggu,0.00 mm,Cerah,None,0
182882,28-Jan-21,7:00,Cimanggu,0.00 mm,Cerah,None,0
182883,28-Jan-21,8:00,Cimanggu,12.00 mm,Lebat,None,1


Number of Missing Value 

In [ ]:
df_tmrain_tmwater.isnull().sum() 

Date                   0
Time                   3
Location               0
Rainfall               3
RainfallStatus         3
WaterLevel        173517
Banjir                 0
dtype: int64

Analyze Dataset

In [ ]:
df_tmrain_tmwater.describe()

,Banjir
count,182885.000000
mean,0.005665
std,0.075051
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [ ]:
df_tmrain_tmwater.cov()

,Banjir
Banjir,0.005633


In [ ]:
df_tmrain_tmwater.corr()

,Banjir
Banjir,1.0


In [ ]:
df_tmrain_tmwater.columns


Index(['Date', 'Time', 'Location', 'Rainfall', 'RainfallStatus', 'WaterLevel',
       'Banjir'],
      dtype='object')

In [ ]:
df_tmrain_tmwater.index

RangeIndex(start=0, stop=182885, step=1)

In [ ]:
df_tmrain_tmwater.values

array([['15-Jul-20', '14:00', 'Sumur Batu', ..., 'Cerah', None, 0],
       ['15-Jul-20', '15:00', 'Sumur Batu', ..., 'Cerah\\', None, 0],
       ['16-Jul-20', '7:00', 'Sumur Batu', ..., 'Cerah', None, 0],
       ...,
       ['28-Jan-21', '7:00', 'Cimanggu', ..., 'Cerah', None, 0],
       ['28-Jan-21', '8:00', 'Cimanggu', ..., 'Lebat', None, 1],
       ['28-Jan-21', '10:00', 'Cimanggu', ..., 'Cerah', None, 0]],
      dtype=object)

In [ ]:
type(df_tmrain_tmwater)

pandas.core.frame.DataFrame

In [ ]:
df_tmrain_tmwater.shape

(182885, 7)

In [ ]:
df_tmrain_tmwater.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182885 entries, 0 to 182884
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Date            182885 non-null  object
 1   Time            182882 non-null  object
 2   Location        182885 non-null  object
 3   Rainfall        182882 non-null  object
 4   RainfallStatus  182882 non-null  object
 5   WaterLevel      9368 non-null    object
 6   Banjir          182885 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 9.8+ MB


In [ ]:
df_tmrain_tmwater['Location']

0         Sumur Batu
1         Sumur Batu
2         Sumur Batu
3         Sumur Batu
4         Sumur Batu
             ...    
182880      Cimanggu
182881      Cimanggu
182882      Cimanggu
182883      Cimanggu
182884      Cimanggu
Name: Location, Length: 182885, dtype: object

In [ ]:
Location_df = df_tmrain_tmwater['Location']

In [ ]:
x=df_tmrain_tmwater['Rainfall','WaterLevel','Time']
y=df_tmrain_tmwater['Banjir']